### Seq2Seq Model Training on Khmer Word to Romanization Word

In [1]:
import numpy as np
import pandas as pd 
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense


2025-10-08 19:24:35.331593: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_kh = pd.read_csv("csv/data_kh.csv", header=None)
data_rom = pd.read_csv("csv/data_rom.csv", header=None)
batch_size = 32  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 7154  # Number of samples to train on.
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()


In [3]:
for input_text in data_kh[0]:
    input_text = str(input_text).strip()
    input_texts.append(input_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

for target_text in data_rom[0]:
    target_text = '\t' + str(target_text).strip() + '\n'
    target_texts.append(target_text)
    for char in str(target_text):
        if char not in target_characters:
            target_characters.add(char)

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 17911
Number of unique input tokens: 76
Number of unique output tokens: 70
Max sequence length for inputs: 30
Max sequence length for outputs: 47


In [4]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

print(input_token_index)
print(target_token_index)

print(encoder_input_data)
print(decoder_input_data)
print(decoder_target_data)

{'\u200b': 0, 'ប': 1, 'រ': 2, 'ខ': 3, 'យ': 4, 'ឆ': 5, 'ផ': 6, 'ឍ': 7, 'ឬ': 8, 'ឡ': 9, 'ី': 10, 'ឿ': 11, 'ោ': 12, 'ហ': 13, 'ស': 14, 'ជ': 15, '៊': 16, 'ឮ': 17, 'ុ': 18, 'ែ': 19, 'ក': 20, '់': 21, 'គ': 22, 'ណ': 23, '័': 24, '?': 25, 'ធ': 26, 'ឭ': 27, 'ច': 28, 'ល': 29, 'ត': 30, 'ឃ': 31, 'ឪ': 32, 'ិ': 33, 'ឺ': 34, 'ង': 35, 'ំ': 36, 'ឦ': 37, 'ដ': 38, 'ួ': 39, 'ទ': 40, '៌': 41, 'ៈ': 42, 'អ': 43, 'ឫ': 44, '៉': 45, 'ៃ': 46, 'ម': 47, 'ា': 48, 'ឲ': 49, '៏': 50, 'ឹ': 51, 'ឋ': 52, 'ះ': 53, 'ឱ': 54, 'វ': 55, 'ឈ': 56, 'េ': 57, 'ូ': 58, 'ៀ': 59, '៍': 60, 'ព': 61, 'ឧ': 62, '្': 63, 'ញ': 64, 'ើ': 65, 'ឯ': 66, 'ថ': 67, 'ឌ': 68, 'ន': 69, 'ឥ': 70, 'ឳ': 71, 'ៅ': 72, 'ឰ': 73, 'ភ': 74, '៎': 75}
{'င': 0, 'u': 1, '\n': 2, 'န': 3, 'a': 4, 'l': 5, 'j': 6, 'C': 7, ' ': 8, 'c': 9, 'T': 10, 'I': 11, 'd': 12, 'ရ': 13, 'ပ': 14, 'ွ': 15, 'ိ': 16, 'ဟ': 17, 'F': 18, 'ဘ': 19, 'ယ': 20, '?': 21, 'v': 22, 'M': 23, 'ဲ': 24, 'x': 25, 'ျ': 26, '/': 27, 'e': 28, 'K': 29, 'r': 30, 'i': 31, 's': 32, 'ဖ': 33, 'p': 34, 'ီ': 35, 'S':

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    #encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
    
# print(encoder_input_data[0])
# print(decoder_input_data[0])
# print(decoder_target_data[0])
print(num_encoder_tokens)

76


In [6]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 123s 269ms/step - accuracy: 0.8310 - loss: 0.6339 - val_accuracy: 0.8196 - val_loss: 0.6645
Epoch 2/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 140s 313ms/step - accuracy: 0.8597 - loss: 0.4745 - val_accuracy: 0.8383 - val_loss: 0.5390
Epoch 3/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 162s 362ms/step - accuracy: 0.8645 - loss: 0.4529 - val_accuracy: 0.8412 - val_loss: 0.5255
Epoch 4/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 138s 308ms/step - accuracy: 0.8661 - loss: 0.4450 - val_accuracy: 0.8410 - val_loss: 0.5205
Epoch 5/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 142s 317ms/step - accuracy: 0.8667 - loss: 0.4393 - val_accuracy: 0.8418 - val_loss: 0.5205
Epoch 6/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 138s 308ms/step - accuracy: 0.8677 - loss: 0.4346 - val_accuracy: 0.8445 - val_loss: 0.5067
Epoch 7/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 134s 299ms/step - accuracy: 0.8686 - loss: 0.4315 - val_accuracy: 0.8449 - val_loss: 0.5052
Epoch 8/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 138s 307ms/step - accuracy: 

In [7]:
# Save model
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save('s2s.h5')

In [8]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [9]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [10]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
#     if str(decoded_sentence).strip() != str(target_texts[seq_index]).strip():
#         print('-')
#         print('KH: ' + input_texts[seq_index] + ', Roman: ' + target_texts[seq_index] + ', predicted: ' + decoded_sentence)
    print('KH: ' + input_texts[seq_index] + ', Roman: ' + target_texts[seq_index] + ', predicted: ' + decoded_sentence)
#     print('Decoded sentence:', decoded_sentence)
#     print('Actual sentence:', target_texts[seq_index])
#     print(str(decoded_sentence).strip() == str(target_texts[seq_index]).strip())
# Get ground truth and remove start/end tokens
    reference = list(target_texts[seq_index].strip("\t\n"))
    hypothesis = list(decoded_sentence.strip("\t\n"))



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
KH: បាន, Roman: 	ban
, predicted: banhchea saam

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/ste